# Create RAG on GCP

## Objectifs

Au cours de ce TP, vous apprendrez à créer un RAG. 

Il s'agit de combiner récupération d'information et génération de texte pour produire des réponses précises en s'appuyant sur des sources externes

On utilisera :
- Vertex AI preview

Import librairies

In [ ]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai
from sklearn.metrics import precision_score, recall_score, f1_score
import time

### Variables

In [ ]:
PROJECT_ID = "A REMPLIR" # Trouvé dans la selection de projet
display_name = "A REMPLIR" # Nom à choisir
paths = ["A REMPLIR"] # Path vers le bucket

In [ ]:
# Initialize Vertex AI API once per session
# Vertex AI RAG Engine ne fonctionne que us-central1 ou europe-west3
vertexai.init(project=PROJECT_ID, location="europe-west3")

### Création corpus du RAG

Avec un model d'embedding

In [ ]:
# Configure embedding model, for example "text-embedding-004".
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-004"
)

rag_corpus = rag.create_corpus(
    display_name=display_name,
    embedding_model_config=embedding_model_config,
)

### Ajouter les fichiers au corpus

In [ ]:
response = rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size="A REMPLIR",  # Optional
    chunk_overlap="A REMPLIR",  # Optional
    max_embedding_requests_per_min="A REMPLIR",  # Optional
)


### Test Retriever

In [ ]:
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
        )
    ],
    text="What IA models are the most efficient ?",
    similarity_top_k="A REMPLIR",  # Optional
    vector_distance_threshold="A REMPLIR",  # Optional
)
print(response)

### Création d'un outil RAG

In [ ]:
# Enhance generation
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,  # Currently only 1 corpus is allowed.
                )
            ],
            similarity_top_k="A REMPLIR",  # Optional
            vector_distance_threshold="A REMPLIR",  # Optional
        ),
    )
)

In [ ]:
# Create a gemini-pro model instance
rag_model = GenerativeModel(
    model_name="A REMPLIR", tools=[rag_retrieval_tool]
)

### Test de la génération

In [ ]:
# Generate response
response = rag_model.generate_content("QUESTION ?")
print(response.text)

## PARTIE EVALUATION

### Installation Libraries :

Ragas => evaluation de RAG

langchain_google_vertexai => Utiliser un modèle dans vertexAI compatible avec Ragas

In [ ]:
!pip install ragas langchain_google_vertexai

### Evaluation du retriever

In [ ]:
import ragas
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness
)

from ragas.evaluation import evaluate
from datasets import Dataset
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

In [ ]:
metrics_eval = [answer_relevancy, # Indique si la réponse générée est pertinente par rapport à la question posée
        faithfulness,             # Vérifie si la réponse est basée uniquement sur les informations fournies dans les contextes récupérés
        context_recall,           # Mesure si le système a récupéré tous les contextes pertinents pour la question
        context_precision,        # Mesure si les contextes récupérés sont pertinents pour la question
        answer_correctness]       # Compare la réponse générée à une réponse de référence pour évaluer sa justesse

### Ragas a besoin d'un dataset avec quatre champs : "question", "ground_truth", "answer", "contexts"

In [ ]:
dict_dataset_eval = {
    "question" : ["What are the models launched by Deepseek ?", "How many Core Contributors ?", "Concerning only MMLU (Pass@1), what are the performance of OpenAI-o1-1217?",],
    "ground_truth" : ["The models lauched by Deekseek are : Deepseek-r1-Zero, Deepseek-r1", "There are 18 Core Contributors", "The performance of OpenAI-o1-1217 on MMLU (Pass@1) is 91.8."],
    "answer" : [],
    "contexts" : []
}

### Fonctions qui remplit le dict_dataset_eval

Récupère la question du dictionnaire, la pose au modèle avec l'outil RAG et stocke les réponses et le contexte dans les listes correspondantes

In [ ]:
def filling_datatset(dict_dataset_eval):
    
    for question in dict_dataset_eval["question"]:
        
        response_retriever = rag.retrieval_query(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,
                )
            ],
            text=question,
            similarity_top_k=5,  # Optional
            vector_distance_threshold=0.5,  # Optional
        )
        
        retrieved_contexts = [context.text for context in response_retriever.contexts.contexts]
        
        dict_dataset_eval["contexts"].append(retrieved_contexts)
        
        response_generator = rag_model.generate_content(question)
        
        dict_dataset_eval["answer"].append(response_generator.text)        
        

In [ ]:
filling_datatset(dict_dataset_eval)

### Ragas a besoin d'un dataset :

In [ ]:
dataset_eval = Dataset.from_dict(dict_dataset_eval)

### Création d'un modèle génératif et d'un modèle d'embeddings

In [ ]:
vertextai_llm = ChatVertexAI(
    model_name="gemini-1.5-flash-001"
)

In [ ]:
vertextai_embeddings = VertexAIEmbeddings(
    model_name="textembedding-gecko"
)

### Evaluation : 

PS : La fonction evaluate() compare les réponses générées avec le RAG et le ground_truth associé avec le modèle vertextai_llm et compare la question et le contexte retrouvé avec vertextai_embeddings.

Le modèle va recevoir beaucoup d'appels par minutes ce qui peut provoquer le message d'erreur : "Retrying langchain_google_vertexai.chat_models._acompletion_with_retry.<locals>._completion_with_retry_inner in 8.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-flash. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.." 

Il faut laisser tourner le run, les appels font se faire.

In [ ]:
ragas_eval_with_gt = evaluate(dataset_eval, metrics_eval, llm=vertextai_llm, embeddings=vertextai_embeddings)

### Résultats :

In [ ]:
ragas_eval_with_gt